## Creando Macros anidadas de Roll20

Primero, he aquí unas pequeñas funciones de prueba para generar tiradas inline

In [1]:
def makelabel(label):
    if label == '':
        return ''
    else:
        return f'[{label}]'
def makecrit(crit):
    if crit == '':
        return ''
    else:
        return f'cs>{crit}'
    
def dice(dicenumber = '1', dicetype = '6', dicebonus = '0', label = '', crit = ''):
    return f'{dicenumber}d{dicetype}{makecrit(crit)}+{dicebonus}{makelabel(label)}'

def inroll(rolls = ['1d8+5[cortante]','3d6+3[fuego]',]):
    formula = rolls[0]
    if len(rolls)>1:
        for roll in rolls[1:]:
            if roll != '':
                formula += '+'+roll        
    return f'[[{formula}]]'

In [2]:
inroll()

'[[1d8+5[cortante]+3d6+3[fuego]]]'

In [3]:
bolafue = dice(3,10,-3,'fuego',18)
inroll([bolafue,bolafue,bolafue])

'[[3d10cs>18+-3[fuego]+3d10cs>18+-3[fuego]+3d10cs>18+-3[fuego]]]'

### La clave del asunto
Esta función reemplaza caracteres especiales de las macros para permitir operaciones anidadas

In [4]:
def leveldeeper(formula):
    formula = formula.replace('&','&amp;')
    formula = formula.replace('\n  ','\n    ')
    formula = formula.replace('|','&#124;')
    formula = formula.replace(',','&#44;')
    formula = formula.replace('}','&#125;')
    return formula

### Querys y menús desplegables
Usando la función anterior, podemos anidar unos dentro de otros a placer

In [5]:
def query(message = 'valor', default = '0'):
    return f'?{{{leveldeeper(message)}|{leveldeeper(default)}}}'

In [6]:
def dropdown(message = 'select', options = ['option1',], labels = []):
    formula = f'?{{{leveldeeper(message)}'
    if len(labels) == len(options):
        for ii in range(len(options)):
            formula += f'|\n  {leveldeeper(labels[ii])}, {leveldeeper(options[ii])}'
    else:
        for ii in range(len(options)):
            formula += f'|\n  {leveldeeper(options[ii])}'
    formula += '}'
    return formula

In [7]:
query('bonus')

'?{bonus|0}'

In [8]:
drop3 = dropdown(options = ['option'+str(ii+1) for ii in range(3)])
print(drop3)

?{select|
  option1|
  option2|
  option3}


In [9]:
print(dropdown(options = [drop3 for ii in range(4)], labels = ['option'+str(ii+1) for ii in range(4)]))

?{select|
  option1, ?{select&#124;
    option1&#124;
    option2&#124;
    option3&#125;|
  option2, ?{select&#124;
    option1&#124;
    option2&#124;
    option3&#125;|
  option3, ?{select&#124;
    option1&#124;
    option2&#124;
    option3&#125;|
  option4, ?{select&#124;
    option1&#124;
    option2&#124;
    option3&#125;}


### Atributos
Permiten referenciar valores de una ficha de personaje en las fórmulas

#### CUIDADO
Los atributos se parsean antes que el resto de la expresión, así que no pueden usarse directamente, debe usarse un placeholder y después sustituirse

In [10]:
def atribute(pj,varname,label = ''):
    return f'[[@{{{pj}|{varname}}}{makelabel(label)}]]'

In [11]:
atribute('Paco', 'str_mod', 'Fuerza')

'[[@{Paco|str_mod}[Fuerza]]]'

## Ejemplo

In [12]:
tipoataque_labels = [
    'Normal',
    'Pericias',
    'Defensa max'
]
tipoataque_options = [
    '',
    '-5[pericias]',
    '-9[combate defensivo]'
]
tipoataque_msgs = [
    'Ataque normal',
    'Ataque con pericia, AC+5',
    'Ataque a la defensiva, AC+7'
]
bonusdrop = dropdown('Tipo de ataque', tipoataque_options, tipoataque_labels)
msgdrop = dropdown('Tipo de ataque (mensaje)', tipoataque_msgs, tipoataque_labels)

In [13]:
print(bonusdrop)

?{Tipo de ataque|
  Normal, |
  Pericias, -5[pericias]|
  Defensa max, -9[combate defensivo]}


# OJO
Es necesario utilizar un placeholder en el lugar del atributo, porque Roll20 parsea en primer lugar los atributos, y se desconfigurarían

In [14]:
atk_var = 'placeholder_atk'
roll_ballesta = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma buena]',
    query('bonus'),
])
msg_ballesta = '\nCrítico: 19-20 x2'

roll_estoque = inroll([
    dice(1,20,atk_var,'',18),
    '1[arma mágica]',
    query('bonus'),
    bonusdrop,
])
msg_estoque = '\nCrítico: 18-20 x2\n' + msgdrop

roll_daga = inroll([
    dice(1,20,atk_var,'',19),
    '1[arma mágica]',
    query('bonus'),
    bonusdrop,
])
msg_daga = '\nCrítico: 19-20 x2\n' + msgdrop

In [15]:
print(roll_daga)

[[1d20cs>19+placeholder_atk+1[arma mágica]+?{bonus|0}+?{Tipo de ataque|
  Normal, |
  Pericias, -5[pericias]|
  Defensa max, -9[combate defensivo]}]]


In [16]:
arma_labels=[
    'Ballesta',
    'Estoque',
    'Daga'
]
arma_options = [
    'Ataque con Ballesta Pesada:\n' + roll_ballesta + msg_ballesta,
    'Ataque con Estoque Mágico:\n' + roll_estoque + msg_estoque,
    'Ataque con Daga Sagrada:\n' + roll_daga + msg_daga,
]
atk_drop = dropdown('Tipo de arma', arma_options, arma_labels)

In [17]:
print(atk_drop)

?{Tipo de arma|
  Ballesta, Ataque con Ballesta Pesada:
[[1d20cs>19+placeholder_atk+1[arma buena]+?{bonus&#124;0&#125;]]
Crítico: 19-20 x2|
  Estoque, Ataque con Estoque Mágico:
[[1d20cs>18+placeholder_atk+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;
    Normal&#44; &#124;
    Pericias&#44; -5[pericias]&#124;
    Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 18-20 x2
?{Tipo de ataque (mensaje)&#124;
    Normal&#44; Ataque normal&#124;
    Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;
    Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;|
  Daga, Ataque con Daga Sagrada:
[[1d20cs>19+placeholder_atk+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;
    Normal&#44; &#124;
    Pericias&#44; -5[pericias]&#124;
    Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 19-20 x2
?{Tipo de ataque (mensaje)&#124;
    Normal&#44; Ataque normal&#124;
    Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;
    Defensa max&#44; Ataque a la defensiv

## Ahora ya sí
Con la macro terminada, se puede sustituir el placeholder por el atributo bueno

In [18]:
print(atk_drop.replace(atk_var, atribute('Goblin', 'strength_mod')))

?{Tipo de arma|
  Ballesta, Ataque con Ballesta Pesada:
[[1d20cs>19+[[@{Goblin|strength_mod}]]+1[arma buena]+?{bonus&#124;0&#125;]]
Crítico: 19-20 x2|
  Estoque, Ataque con Estoque Mágico:
[[1d20cs>18+[[@{Goblin|strength_mod}]]+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;
    Normal&#44; &#124;
    Pericias&#44; -5[pericias]&#124;
    Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 18-20 x2
?{Tipo de ataque (mensaje)&#124;
    Normal&#44; Ataque normal&#124;
    Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;
    Defensa max&#44; Ataque a la defensiva&amp;#44; AC+7&#125;|
  Daga, Ataque con Daga Sagrada:
[[1d20cs>19+[[@{Goblin|strength_mod}]]+1[arma mágica]+?{bonus&#124;0&#125;+?{Tipo de ataque&#124;
    Normal&#44; &#124;
    Pericias&#44; -5[pericias]&#124;
    Defensa max&#44; -9[combate defensivo]&#125;]]
Crítico: 19-20 x2
?{Tipo de ataque (mensaje)&#124;
    Normal&#44; Ataque normal&#124;
    Pericias&#44; Ataque con pericia&amp;#44; AC+5&#124;
    Defe